In [1]:
#Dependencies
import pandas as pd


In [2]:
#Extracting Files (Kdrama)
kdrama_file = "./Data/imdb.csv"
kdrama_df = pd.read_csv(kdrama_file)
kdrama_df.head()

,kdrama_name,imdb_rating,imdb_users,imdb_description
0,100 Days My Prince,7.7,"2,188","Upon losing his memory, a crown prince encount..."
1,12 Years Promise,7.3,181,A pregnant teen is forced by her family to lea...
2,18 Again,8.2,"1,847",A 37-year-old man on the verge of being divorc...
3,365: Repeat the Year,8.0,656,A story where ten people get the chance to go ...
4,7th Grade Civil Servant,6.3,145,A romantic comedy about a spy couple who hides...


In [4]:
#Cleaning Files (kdrama)
cleaned_kdrama_df = kdrama_df[["kdrama_name", "imdb_rating"]].copy()
cleaned_kdrama_df.sort_values(by=['imdb_rating'], inplace=True, ascending=False)
cleaned_kdrama_df.head()

,kdrama_name,imdb_rating
244,If You Wish Upon Me,9.5
452,Reply 1988,9.2
518,Standby,9.2
158,Extraordinary Attorney Woo,9.1
373,My Mister,9.1


In [6]:
#Genres
kdrama_genre_file = "./Data/genres.csv"
kdrama_genre_file_df = pd.read_csv(kdrama_genre_file)
kdrama_genre_file_df.head()

,kdrama_name,genre
0,100 Days My Prince,historical
1,100 Days My Prince,comedy
2,100 Days My Prince,romance
3,100 Days My Prince,drama
4,12 Signs of Love,comedy


In [7]:
inner_join_df= pd.merge(cleaned_kdrama_df, kdrama_genre_file_df, on='kdrama_name', how='inner')
inner_join_df_above_8 = inner_join_df[inner_join_df['imdb_rating'] > 8] 
inner_join_df_above_8

,kdrama_name,imdb_rating,genre
0,If You Wish Upon Me,9.5,life
1,If You Wish Upon Me,9.5,drama
2,If You Wish Upon Me,9.5,medical
3,Reply 1988,9.2,comedy
4,Reply 1988,9.2,romance
...,...,...,...
414,Circle,8.1,scifi
415,Heartless City,8.1,action
416,Heartless City,8.1,thriller
417,Heartless City,8.1,romance


In [8]:
inner_join_df_above_8.value_counts("genre")

genre
 drama            66
 romance          52
 life             23
 comedy           22
 melodrama        21
romance           21
 mystery          20
comedy            18
thriller          18
action            17
 family           15
 political        12
historical        11
 fantasy           9
 crime             9
 historical        8
 law               7
business           7
 thriller          6
 medical           6
 psychological     5
mystery            5
 sitcom            5
 scifi             5
psychological      5
 youth             4
life               4
drama              3
law                2
military           2
 sports            2
friendship         2
 supernatural      2
horror             1
music              1
 military          1
 horror            1
 business          1
dtype: int64